In [87]:
import math
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import pandas as pd


all_reviews_name = []
all_reviews_link = []
all_reviews_date = []
all_reviews_film = []
all_opinions = []
all_press_names = []
with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    num_reviews_xpath = '//div[@class="pagesFromTo"]'
    press_name_xpath = '//h1[@class="breadcrumbs__head"]'
    positives_xpath = '//div[contains(@class, "item positive")]'
    negatives_xpath = '//div[contains(@class, "item negative")]'
    for press_id in range(155, 156 + 1):
        driver.get(f'https://www.kinopoisk.ru/press/source/{press_id}')
        press_name =  driver.find_elements(By.XPATH, press_xpath)[1].text
        try:
            num_reviews = driver.find_element(By.XPATH, num_reviews_xpath).text.split()[2]
            num_pages = math.ceil(int(num_reviews) / 25)
        except NoSuchElementException:
            num_pages = 1
        for page_number in range(1, num_pages + 1):
            driver.get(f'https://www.kinopoisk.ru/press/source/{press_id}/page/{page_number}/#list')
            for press_opinion_xpath in [positives_xpath, negatives_xpath]:
                reviews_name = driver.find_elements(By.XPATH, press_opinion_xpath + '//div[@class="name"]/a')
                all_reviews_name.extend([review.text for review in reviews_name])
                reviews_link = driver.find_elements(By.XPATH, press_opinion_xpath + '//div[@class="fullText"]/a')
                all_reviews_link.extend([review.get_attribute('href') for review in reviews_link])
                reviews_date = driver.find_elements(By.XPATH, press_opinion_xpath + '//div[@class="date"]')
                all_reviews_date.extend([review.text for review in reviews_date])
                reviews_film = driver.find_elements(By.XPATH, press_opinion_xpath + '//div[@class="subject"]/a')
                all_reviews_film.extend([review.get_attribute('href') for review in reviews_film])
                press_opinions = [re.search(r'positive|negative', press_opinion_xpath)[0] for _ in range(len(reviews_name))]
                all_press_names.extend(press_name for _ in range(len(reviews_name)))
                all_opinions.extend(press_opinions)
    print(all_reviews_date, all_reviews_film, all_reviews_link, all_reviews_name, all_opinions, all_press_names)

['20 мая 2022', '16 марта 2022', '5 декабря 2021', '12 декабря 2021'] ['https://www.kinopoisk.ru/film/4642708/press/', 'https://www.kinopoisk.ru/film/1108689/press/', 'https://www.kinopoisk.ru/film/4445150/press/', 'https://www.kinopoisk.ru/film/4357563/press/'] ['https://t.me/cinemarticle/8746', 'https://t.me/cinemarticle/8280', 'https://t.me/cinemarticle/7764', 'https://t.me/cinemarticle/7822'] ['Екатерина Высоцких', 'Екатерина Высоцких', 'Екатерина Высоцких', 'Екатерина Высоцких'] ['positive', 'positive', 'positive', 'negative'] ['Cinemarticle', 'Cinemarticle', 'Cinemarticle', 'Cinemarticle']
['20 мая 2022', '16 марта 2022', '5 декабря 2021', '12 декабря 2021', '24 июня 2022', '23 мая 2022', '6 апреля 2022', '18 апреля 2022'] ['https://www.kinopoisk.ru/film/4642708/press/', 'https://www.kinopoisk.ru/film/1108689/press/', 'https://www.kinopoisk.ru/film/4445150/press/', 'https://www.kinopoisk.ru/film/4357563/press/', 'https://www.kinopoisk.ru/film/4397114/press/', 'https://www.kinopoi

In [3]:

data = [[1, 2, 3], [4, 5, 6]]
data.append([7, 8, 9])
df = pd.DataFrame(data, columns=['1', '2', '3'])
df

,1,2,3
0,1,2,3
1,4,5,6
2,7,8,9
